In [1]:
!gdown 14w2Vpqlhrid_ufJEr_RZoMDZ9ZfyECur

Downloading...
From: https://drive.google.com/uc?id=14w2Vpqlhrid_ufJEr_RZoMDZ9ZfyECur
To: /content/tweets-data.zip
100% 409k/409k [00:00<00:00, 107MB/s]


In [2]:
!unzip tweets-data.zip

Archive:  tweets-data.zip
  inflating: tweets-data/demo.csv    
  inflating: tweets-data/disaster_classifier.sav  
  inflating: tweets-data/fire-test-wtite-firebase-adminsdk-roj7r-761ecacfaf.json  
  inflating: tweets-data/temp_file.csv  
  inflating: tweets-data/temp_file.old.csv  
 extracting: tweets-data/x-authen.json  


In [ ]:
!pip install pythainlp -q
!pip install firebase_admin -q
!pip install schedule -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 58.4 MB/s eta 0:00:00


# Cell นี้ run แค่ครั้งเดียว
** หากเกิด error ให้ลบรันไทม์และรันใหม่อีกครั้ง

In [ ]:
from collections import Counter
import subprocess
import requests as req
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re
import os
from pathlib import Path
from pythainlp.phayathaibert.core import NamedEntityTagger
import pickle
from firebase_admin import credentials,firestore
import firebase_admin

!npx playwright install
cred = credentials.Certificate("/content/tweets-data/fire-test-wtite-firebase-adminsdk-roj7r-761ecacfaf.json")
firebase_admin.initialize_app(cred)
def model(text):
    disaster_model = pickle.load(open('/content/tweets-data/disaster_classifier.sav','rb'))
    prediction = disaster_model.predict(text,k=2)
    return prediction

def get_location(text):
    tagger = NamedEntityTagger()
    location = []
    for i in tagger.get_ner(text):
      if i[1] == 'LOCATION' and (i[0] != '' or i[0] != ' '):
        # print('lo')
        # print(i[0])
        location.append(i[0])
      # else :
      #   print('no lo')
      #   location.append('no location')
    return location

def get_filenames(directory):
  file_names = []
  for filename in os.listdir(directory):
    if os.path.isfile(os.path.join(directory, filename)):
      file_names.append(filename)
  return file_names

def moniter_hashtag():
    web = 'https://twitter-trends.iamrohit.in/thailand'
    res = req.get(web).content
    soup = BeautifulSoup(res, 'html.parser')
    ul2 = soup.find_all('a',class_="tweet")
    info = []
    for i in ul2 :
        if model(i.text) != 'normal':
          info.append([i.text,model(i.text),get_location(i.text)])
    return info

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')  # Compiled URL pattern
    return url_pattern.sub('', text)  # Efficiently replace URLs with empty string

def cleaning_data(df):
  df.drop_duplicates(subset='full_text', inplace=True)
  col_name_list = ['conversation_id_str',	'created_at',	'favorite_count',	'full_text',	'id_str', 'quote_count',	'reply_count', 'retweet_count',	'user_id_str']
  all_cols = list(df.columns)
  cols_to_delete = [col for col in all_cols if col not in col_name_list]
  df.drop(cols_to_delete, axis=1, inplace=True)
  df['full_text'] = df['full_text'].apply(remove_urls)
  return df['full_text']


def save_data(status,location,title):
  data = {
    'header':status,
    'location':location,
    'title':title
    }
  db = firestore.client()
  user_ref = db.collection(u'info').document(u'notify')
  user_ref.set(data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.26M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.4M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


npx: installed 2 in 3.403s
╔═══════════════════════════════════════════════════════════════════════════════╗
║ WARNING: It looks like you are running 'npx playwright install' without first ║
║ installing your project's dependencies.                                       ║
║                                                                               ║
║ To avoid unexpected behavior, please install your dependencies first, and     ║
║ then run Playwright's install command:                                        ║
║                                                                               ║
║     npm install                                                               ║
║     npx playwright install                                                    ║
║                                                                               ║
║ If your project does not yet depend on Playwright, first install the          ║
║ applicable npm package (most commonly @playwright/test), and         

In [ ]:
import json

with open("/content/tweets-data/x-authen.json", "r") as f:
  data = json.load(f)
key = data["key"]

'6cc8ec184744fc331bef82c54312cf1c5578a372'

In [ ]:
def run():
  temp = ''
  n = moniter_hashtag()
  if len(n) > 0:
      for i in range(len(n)):
          # path = htg + '.csv'
          command = f'npx tweet-harvest@latest -s "{n[i][0]}" -l "10" --token "{key}"'
          subprocess.run(command, shell=True)
          filenames = get_filenames("/content/tweets-data")
          for h in filenames:
            if n[i][0] in h:
              temp = h
          tweets = pd.read_csv("/content/tweets-data/"+temp)
          print('Get tweets from keyword ' + n[i][0] + ' success!!')
          data = cleaning_data(tweets)
          data = list(data)
          dis = []
          loc = []
          hsg = n[i][0]
          for text in data:
              if model(text) != 'normal':
                  dis.append(model(text))
                  loc.append(get_location(text))
          if Counter(dis).most_common(1)[0][0] == n[i][0] and Counter(dis).most_common(1)[0][1]/len(data) > 0.7:
            if n[i][0] == 'fire':
              save_data(n[i][0],loc[0],f'แจ้งเตือนไฟไหม้ที่{loc[0]}')
            elif n[i][0] == 'shooting':
              save_data(n[i][0],loc[0],f'แจ้งเตือนกราดยิงที่{loc[0]}')
            else :
              save_data(n[i][0],loc[0],f'แจ้งเตือนแผ่นดินไหวที่{loc[0]}')
          else:
            print('safe')

# ใช้งานจริง

** cell นี้ใช้เพื่อทำงานจริงเมื่อต้องการหยุดใช้งานให้หยุดรันแต่จะแสดง error แต่ไม่มีปัญหาสามารถข้ามไปทดลองรัน cell อื่นต่อได้

In [ ]:
import schedule
import time
schedule.every(5).minutes.do(run)
while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 

# Demo การใช้งานเมื่อเจอเหตุการณ์อันตราย

In [ ]:
demo = pd.read_csv('/content/tweets-data/demo.csv')

In [ ]:
demo['Label'] = demo['Label'].str.replace(" ",'')

In [ ]:
demo

,Tweets,Label
0,ตอนนี้มีไฟไหม้ที่โกดังเก็บฟางในตำบลพุกร่าง อำเ...,fire
1,เกิดแผ่นดินไหวขนาดเล็กในพื้นที่จังหวัดแพร่,earthquake
2,มีเหตุการณ์กราดยิงเกิดขึ้นที่ห้างสรรพสินค้าพาร...,shooting
3,ไฟไหม้โรงงานกิ่งแก้วในจังหวัดสมุทรปราการ,fire
4,เกิดแผ่นดินไหวขนาดใหญ่เขย่าจังหวัดเชียงใหม่และ...,earthquake
5,เกิดเหตุการณ์กราดยิงในพื้นที่อำเภอสายบุรี จังห...,shooting
6,ไฟไหม้บริเวณโรงงานในจังหวัดพระนครศรีอยุธยา,fire
7,เกิดแผ่นดินไหวขนาดเล็กในพื้นที่จังหวัดพะเยา,earthquake
8,มีเหตุการณ์กราดยิงเกิดขึ้นที่ร้านสะดวกซื้อในจั...,shooting
9,ไฟไหม้ป่าในเขตอุทยานแห่งชาติเขาใหญ่ จังหวัดนคร...,fire


In [ ]:
fire = demo[demo['Label']=='fire']
shooting = demo[demo['Label']=='shooting']
earthquake = demo[demo['Label']=='earthquake']

In [ ]:
text = demo.iloc[1]['Tweets']

In [ ]:
text = "ไฟไหม้บางนาแผ่นไหวลาดกระบัง"

In [ ]:
if model(text) != 'normal':
    loc = ''.join(get_location(text))
    if 'ที่' in loc:
      loc = loc.replace('ที่','')
    if model(text) == 'fire':
      save_data('ไฟไหม้',loc,f'แจ้งเตือนไฟไหม้ที่{loc}')
    elif model(text) == 'shooting':
      save_data('กราดยิง',loc,f'แจ้งเตือนกราดยิงที่{loc}')
    else :
      save_data('แผ่นดินไหว',loc,f'แจ้งเตือนแผ่นดินไหวที่{loc}')

tokenizer_config.json:   0%|          | 0.00/144k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/2.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
model("ไฟไหม้บางนาแผ่นไหวลาดกระบัง")

'fire'